Давайте решим следующую задачу.<br>
Необходимо написать робота, который будет скачивать новости с сайта Лента.Ру и фильтровать их в зависимости от интересов пользователя. От пользователя требуется отмечать интересующие его новости, по которым система будет выделять области его интересов.<br>


Начнем с загрузки новостей. Для этого нам потребуется метод requests.get(url). Библиотека requests предоставляет серьезные возможности для загрузки информации из Интернет. Метод get получает URL стараницы и возвращает ее содержимое. В нашем случае результат будет получаться в формате html. <br>
Загрузим необходимые библиотеки.

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

Теперь попробуем загрузить страницу новостей.

In [2]:
# Для пробы получаем первую страницу сайта.
requests.get("https://lenta.ru/")

<Response [200]>

Метод <i>requests.get()</i> возвращает объект Response, который содержит большое количество различной информации о загруженной (или незагруженной) странице. В краткой форме отображается только результат выполения запроса. В нашем случае это 200, нет ошибки.<br> 
Посмотрим что результат содержит еще.

In [3]:
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")

In [4]:
dir(resp)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [5]:
%%time
# %%time - Магия Jupyter - замеряет время выполнения ячейки. Должно быть первой строчкой в ячейке.
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie lid=vAsAACoiG2i7QdSFAV3cAgB= for .lenta.ru/>, <Cookie lids=482641BBC88A7238 for .lenta.ru/>]>
time to download: 0:00:00.179992
page encoding utf-8
Server response:  200
Is everything ok?  True
Page's URL:  https://lenta.ru/news/2018/08/24/clon/
CPU times: user 41.8 ms, sys: 2.12 ms, total: 43.9 ms
Wall time: 182 ms


In [6]:
for c in resp.cookies:
    print(c)

<Cookie lid=vAsAACoiG2i7QdSFAV3cAgB= for .lenta.ru/>
<Cookie lids=482641BBC88A7238 for .lenta.ru/>


Но самое для нас интересное хранится в поле <i>text</i>, которое содержит собственно текст html-страницы.

In [7]:
#Берем первые 1000 символов новости.
resp.text[:1000]

'<!DOCTYPE html><html lang="ru"><head><title>В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru</title><meta charset="utf-8" /><meta content="#292929" name="theme-color" /><link href="https://lenta.ru/rss/google-newsstand/main/" rel="alternate" type="application/rss+xml" /><link href="https://lenta.ru/news/2018/08/24/clon/" rel="canonical" /><link href="/manifest.json" rel="manifest" /><link rel="icon" type="image/x-icon" href="https://icdn.lenta.ru/favicon.ico" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-256x256.png" size="256x256" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-192x192.png" size="192x192" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-152x152.png" size="152x152" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-120x120.png" size=

Количество служебной информации в странице явно превышает объем текста новости. У нас есть два пути: либо использовать библиотеку BeautyfulSoup для получения текста статьи, либо получить текст с использованием регулярных выражений.

Опробуем первый путь. Документация на библиотеку BeautyfulSoup находится <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">здесь</a>.

В ячейке ниже мы создаем объект BeautifulSoup, передаем в него текст html-страницы и сообщаем, что разбирать его надо при помощи библиотеки `html5lib`. Далее просим отдать текст страницы без html-тегов.

In [8]:
BeautifulSoup(resp.text, "html5lib").get_text()[:1000]

'В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru{"@context":"http://schema.org","@type":"NewsArticle","headline":"В Сибири нашли подходящих для клонирования древних животных","description":"Палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Подобные находки имеют большое количество сохранившейся ДНК, которая подходит для клонирования.","name":"В Сибири нашли подходящих для клонирования древних животных","alternativeHeadline":"","url":"https://lenta.ru/news/2018/08/24/clon/","mainEntityOfPage":{"@type":"WebPage","@id":"https://lenta.ru/news/2018/08/24/clon/"},"thumbnailUrl":"https://icdn.lenta.ru/images/2018/08/24/13/20180824130540171/detail_fb4ef24e26e8448a2a23747f000a4489.jpg","dateCreated":"2018-08-24T15:22:00+03:00","datePublished":"2018-08-24T15:22:00+03:00","dateModified"

Да, убрать html-теги получилось. Но их содержимое осталось, в том числе и скрипты.<br>
Опробуем другой путь. Весь текст обычно оформляется тегом параграфа - &lt;p&gt;. Выберем весь текст из этих тегов. Заодно выберем и заголовок статьи, оформленный при помощи \<h1\>.

In [9]:
# Получили объект BeautifulSoup и скормили ему текст страницы.
bs = BeautifulSoup(resp.text, "html5lib") 
# Вот таким образом можно попросить отдать первый тег, отмеченный как h1. Вместо h1 можно написать любой другой тег.
title = bs.h1.text
# Получаем все параграфы (тег p), берем их текст без тегов и склеиваем в один текст.
text = "\n".join([p.text for p in bs.find_all("p")])
print(title, "\n-----\n", text)
print(bs("p"))

В Сибири нашли подходящих для клонирования древних животных 
-----
 Фото: AP
Российские палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Об этом сообщается в пресс-релизе на Phys.org.
Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Таким образом, находка является потенциально пригодной для клонирования животного.
У найденного ископаемого, относящегося к вымершему виду Equus lenensis, сохранились кожа, шерсть, копыта, хвост и внутренние органы. Возраст жеребенка на момент смерти составлял примерно 2-3 месяца. Причиной смерти, вероятно, является попадание в какую-то «ловушку» естественного происхождения, поскольку видимых повреждений на теле не было.
У трупа были взяты образцы шерсти и биологических жидкостей для тщательного генетического анализа. По словам исследователей, на данный момент это самые хорошо сохранившиеся из всех останков древних лошадей.
В 2015 году в Я

Теперь напишем функцию, которая выгружает все новости за сутки. <br>
Обратим внимание, что для сайта Lenta.ru можно написать адрес в формате lenta.ru/ГГГГ/ММ/ДД/ (год, месяц, день) и получить все новости за этот день. Попробуем получить все адреса с такой страницы.

In [10]:
# Идем на страницу, получаем ее текст, отдаем в BeautifulSoup, ищем все теги ссылок - а.
BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html5lib").find_all("a")[:20]

[<a class="menu__nav-link _is-extra" href="/">Главное</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/russia/">Россия</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/world/">Мир</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/ussr/">Бывший СССР</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/economics/">Экономика</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/forces/">Силовые структуры</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/science/">Наука и техника</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/culture/">Культура</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/sport/">Спорт</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/media/">Интернет и СМИ</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/style/">Ценности </a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/travel/">Путешествия</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/life/">Из жизни</a>,
 <a class="menu__nav-lin

Кажется, это опять немного не то, что нам нужно. Мы получили все ссылки, находящиеся на боковом меню, ссылки на события сегодняшнего дня и другие ненужные нам вещи. <br>
Смотрим в содержимое html-страницы и обращаем внимание, что все интересные нам ссылки оформлены как заголовки третьего уровня - &lt;h3&gt;. Извлечем все такие фрагменты, а потом извлечем собственно адреса, помеченные атрибутом href тега &lt;a&gt;.

In [11]:
# Теперь выделим только то, что взято в тег h3.
beau = BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html5lib")
tags = beau.find_all("a", attrs={'class': 'card-full-news'})
# Формируем список ссылок. Для этого берем первую (кстати, единственную) ссылку из каждого выделенного
# фрагмента, у нее берем значение параметра href. Так как ссылки внутренние, добавляем к ним адрес сайта.
links = ["https://lenta.ru"+l["href"] for l in tags]
print(links)

['https://lenta.ru/news/2018/08/25/potomu/', 'https://lenta.ru/news/2018/08/25/rastvorova/', 'https://lenta.ru/news/2018/08/25/razvedka/', 'https://lenta.ru/news/2018/08/25/poezd/', 'https://lenta.ru/news/2018/08/25/bolton/', 'https://lenta.ru/news/2018/08/25/nur/', 'https://lenta.ru/news/2018/08/25/firetornado/', 'https://lenta.ru/news/2018/08/25/drake/', 'https://lenta.ru/news/2018/08/25/merkel/', 'https://lenta.ru/news/2018/08/25/serena/', 'https://lenta.ru/news/2018/08/25/python/', 'https://lenta.ru/news/2018/08/25/ukrain/', 'https://lenta.ru/news/2018/08/25/vsetaki_pustili/', 'https://lenta.ru/news/2018/08/25/boycott_curtis/', 'https://lenta.ru/news/2018/08/25/twitterrr/', 'https://lenta.ru/news/2018/08/25/milliardy/', 'https://lenta.ru/news/2018/08/25/otrajenie/', 'https://lenta.ru/news/2018/08/25/dope_ioc/', 'https://lenta.ru/news/2018/08/25/edro/', 'https://lenta.ru/news/2018/08/25/galaktika_v_opasnosti/', 'https://lenta.ru/news/2018/08/25/seady_steady/', 'https://lenta.ru/news

Если теперь написать функцию, которая будет перебирать все адреса и получать из них тексты новостей, то мы получим все новости за определенные сутки. Но это мы сделаем чть позже, а пока просто оформим код загрузки статьи в виде функции.

In [13]:
# Загрузка статьи по URL.
def getOneLentaArticle(url):
    """ getLentaArticle gets the body of an article from Lenta.ru"""
    # Получает текст страницы.
    resp=requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs=BeautifulSoup(resp.text, "html5lib") 
    # Получаем заголовок статьи.
    aTitle=bs.h1.text.replace("\xa0", " ")
    # Получаем текст статьи.
    anArticle=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
    # Возвращаем кортеж из заголовка и текста статьи.
    return aTitle, anArticle


### XPath
[Ссылка 1](https://habr.com/ru/post/526774/)

[Ссылка 2](https://habr.com/ru/post/464897/)

А теперь сделаем то же самое при помощи XPath.

In [14]:
from lxml import html

In [15]:
page = requests.get('https://lenta.ru/news/2021/02/27/apple_effect/')

In [23]:
tree = html.fromstring(page.text)
print(tree.xpath(".//h1")[0].text_content())
print(tree.xpath(".//a[contains(@class, 'topic-header__time')]")[0].text_content().strip())
print(tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip(), '\n')
print(tree.xpath(".//meta[@name='og:description']")[0].get("content"), '\n')

for p in tree.xpath(".//div[@class='topic-body__content']/p"):
    print(p.text_content())

Обнаружен неожиданный эффект от употребления яблок
15:35, 27 февраля 2021
Соня Кошечкина (Редактор) 

Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний  обнаружили неожиданный эффект от употребления яблок. Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. 

Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм т

А теперь та же страница, но через BeautyfulSoup.

In [24]:
souped = BeautifulSoup(page.text)

title = souped("h1")[0].get_text()
when = souped.find_all("a", attrs={'class': 'topic-header__time'})[0].get_text().strip()
author = souped.find_all("div", attrs={'class': 'topic-authors'})[0].get_text()
description = souped.find_all("meta", attrs={'name': 'og:description'})[0]["content"]

for p in souped.find_all("div", attrs={'class': 'topic-body__content'})[0]("p", attrs={'class':"topic-body__content-text"}): 
    print(p.get_text())
    


Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.
Ранее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и поэтому яблоко желательно съедать вместе с огрызком.


In [25]:
title, when, author

('Обнаружен неожиданный эффект от употребления яблок',
 '15:35, 27 февраля 2021',
 'Соня Кошечкина (Редактор)')

### Cookies

Библиотека также позволяет работать с куки. 

In [26]:
le = requests.get("https://lenta.ru")
print(le.cookies)

<RequestsCookieJar[<Cookie lid=vAsAAM8jG2jVQGezAXzZAgB= for .lenta.ru/>, <Cookie lids=482240D5C8F3C6C8 for .lenta.ru/>]>


In [23]:
c1 = le.cookies
# c1
c1.set('sdjkfsa', '123445') 
# c1

Cookie(version=0, name='sdjkfsa', value='123445', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [16]:
coo = requests.cookies.RequestsCookieJar()
coo.set("asfdasfd", "12341234")
res = requests.get("https://lenta.ru", cookies=coo)

### Заголовки
При работе с http, мы обмениваемся с сервером заголовками. Например, сервер рассказывает свою версию, какого вида данные он нам вернул, какие он поддерживает протоколы.

In [17]:
res.headers

{'Server': 'nginx', 'Date': 'Mon, 22 Apr 2024 08:49:11 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=50', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Download-Options': 'noopen', 'X-Permitted-Cross-Domain-Policies': 'none', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'P3P': 'CP="This Is Potato!", CP="NON DSP NID ADMa DEVa TAIa PSAa PSDa OUR IND UNI COM NAV"', 'ETag': 'W/"3148ede84741eed0716be6a6e5206388"', 'Cache-Control': 'max-age=0, private, must-revalidate', 'X-Request-Id': 'fdf62e20-b253-47f5-9ca3-31d6c7dc7aa6', 'X-Runtime': '0.761965', 'Set-Cookie': 'is_mobile=0; path=/; domain=.lenta.ru, lids=48225E8D8921F996;path=/;Max-Age=1800;domain=.lenta.ru, lid=vAsAAIckJmaNXrKLATOQAwB=; expires=Thu, 31-Dec-37 23:55:55 GMT; domain=.lenta.ru; path=/', 'Access-Control-Allow-Origin': '*', 'Content-Encoding': 'br'}

Мы тоже можем ему передать заголовки с интересной для нас информацией (можно посмотреть в отладочной информации браузера). Например, наша программа может начать представляться другим браузером.

In [47]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 YaBrowser/20.9.3.136 Yowser/2.5 Safari/537.36'}

response = requests.get('https://lenta.ru', headers=headers)

### А теперь случай посложнее

Вот здесь  
https://warszawa.wyborcza.pl/warszawa/7,54420,31835524,tak-metro-zmienilo-warszawe-pokazujemy-zdjecia-przed-i-po-budowie.html  
народ никак не хочет отдавать страничку. Можно использовать [Selenium](https://selenium-python.readthedocs.io/), он даже не будет открывать окно, если его правильно попросить. Но это будет значительно дороже по нагрузке на процессор и трафику, потому что грузиться будет не только страница, но и всё подряд. С другой стороны, Selenium запускает все скрипты, чтобы получить все оттображаемые данные, или запустить нужный нам скрипт.  
Но нам хочется обойтись без него.

In [48]:
ww = requests.get("https://warszawa.wyborcza.pl/warszawa/7,54420,31835524,tak-metro-zmienilo-warszawe-pokazujemy-zdjecia-przed-i-po-budowie.html")
souped = BeautifulSoup(ww.text)
souped("div")[2: 5]

[<div id="info-adblock">
 <h1>Wyłącz AdBlocka/uBlocka</h1>
 <p class="head">Aby czytać nasze artykuły wyłącz AdBlocka/uBlocka lub dodaj wyjątek dla naszej domeny.</p>
 <p class="lead">Spokojnie, dodanie wyjątku nie wyłączy blokowania reklam.</p>
 </div>,
 <div id="info-ups">
 <h1>Ups!</h1>
 <p class="lead">Nieznany błąd - nie można wyświetlić strony</p>
 </div>,
 <div class="advertHolder adHolder" id="adHolder" style="display: block"></div>]

Ругается на adBlocker и ничего не показывает. Но в браузере не показывает, то есть дело не в нём. Может бытьт надо вернутьт куки, чтобы они подумали, что мы в браузере, и представиться браузером?

In [49]:
ww.cookies

<RequestsCookieJar[Cookie(version=0, name='SquidLocalUID', value='8b92ec40c39af58d00495262', port=None, port_specified=False, domain='.wyborcza.pl', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1841219038, discard=False, comment=None, comment_url=None, rest={'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='SsoSessionPermanent', value='a9cb6d2e403b8da1d59e5adb4bc342ef350a7c9f97fe0721bf72e761f854cf23', port=None, port_specified=False, domain='.wyborcza.pl', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1904291038, discard=False, comment=None, comment_url=None, rest={'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='GW_SID', value='7AA1AC23EF1FA623530E2585E30DDD84.tomwybo52', port=None, port_specified=False, domain='warszawa.wyborcza.pl', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True

In [50]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 YaBrowser/20.9.3.136 Yowser/2.5 Safari/537.36'}
ccc = ww.cookies

ww = requests.get("https://warszawa.wyborcza.pl/warszawa/7,54420,31835524,tak-metro-zmienilo-warszawe-pokazujemy-zdjecia-przed-i-po-budowie.html",
                  headers=headers,
                  cookies=ccc
                 )
souped = BeautifulSoup(ww.text)
souped("div")

[<div class="msg-container">
 <div id="message">
 <img src="https://bi.gazeta.pl/im/3/17117/m17117193.png"/>
 <div id="info-adblock">
 <h1>Wyłącz AdBlocka/uBlocka</h1>
 <p class="head">Aby czytać nasze artykuły wyłącz AdBlocka/uBlocka lub dodaj wyjątek dla naszej domeny.</p>
 <p class="lead">Spokojnie, dodanie wyjątku nie wyłączy blokowania reklam.</p>
 </div>
 <div id="info-ups">
 <h1>Ups!</h1>
 <p class="lead">Nieznany błąd - nie można wyświetlić strony</p>
 </div>
 </div>
 <div class="advertHolder adHolder" id="adHolder" style="display: block"></div>
 </div>,
 <div id="message">
 <img src="https://bi.gazeta.pl/im/3/17117/m17117193.png"/>
 <div id="info-adblock">
 <h1>Wyłącz AdBlocka/uBlocka</h1>
 <p class="head">Aby czytać nasze artykuły wyłącz AdBlocka/uBlocka lub dodaj wyjątek dla naszej domeny.</p>
 <p class="lead">Spokojnie, dodanie wyjątku nie wyłączy blokowania reklam.</p>
 </div>
 <div id="info-ups">
 <h1>Ups!</h1>
 <p class="lead">Nieznany błąd - nie można wyświetlić strony<

Не помогло. Может дело и не в куки. Или мы что-то не от вернули. Пойдём посморим на страничку целиком.

In [51]:
ww.text[:1000]

'\n<!DOCTYPE html>\n<html>\n<head>\n    <meta charset="utf-8"/>\n    <title>Wyborcza.pl</title>\n    <noscript>\n        <meta http-equiv="Refresh" content="0; URL=https://warszawa.wyborcza.pl/warszawa/7,54420,31835524,tak-metro-zmienilo-warszawe-pokazujemy-zdjecia-przed-i-po-budowie.html?squid_js=false"/>\n    </noscript>\n    <meta http-equiv="cache-control" content="max-age=0"/>\n    <meta http-equiv="cache-control" content="no-cache"/>\n    <meta http-equiv="expires" content="0"/>\n    <meta http-equiv="expires" content="Tue, 01 Jan 1980 1:00:00 GMT"/>\n    <meta http-equiv="pragma" content="no-cache"/>\n    <link rel="shortcut icon" href="https://static.im-g.pl/aliasy/foto/wyborcza/favicon.ico">\n    <style type="text/css">\n        body {\n            font-family: Arial, sans-serif;\n            font-size: 13px;\n        }\n\n        h1 {\n            font-size: 16px;\n        }\n\n        a {\n            color: #146cb4;\n            text-decoration: none;\n        }\n\n        

И прямо сверху - вот оно! 
```HTML
 <noscript>\n        <meta http-equiv="Refresh" content="0; URL=https://warszawa.wyborcza.pl/warszawa/7,54420,31835524,tak-metro-zmienilo-warszawe-pokazujemy-zdjecia-przed-i-po-budowie.html?squid_js=false"/>\n    </noscript>
```

То есть страница сама в себе хранит информацию о том, куда надо сделать редирект. 
То есть можно просто загрузить новость с этой страницы и всё.  
Исследование показывает, что страница отличается наличием параметра `?squid_js=false"`, то есть можно просто добавлять его к полученным адресам и всё.

In [52]:
ww = requests.get("https://warszawa.wyborcza.pl/warszawa/7,54420,31835524,tak-metro-zmienilo-warszawe-pokazujemy-zdjecia-przed-i-po-budowie.html?squid_js=false")
souped = BeautifulSoup(ww.text)
souped("h1")[0].get_text(), \
souped("div", attrs={'class': 'article--content'})[0].get_text()

('Tak metro zmieniło Warszawę. Pokazujemy zdjęcia tych samych miejsc przed i po budowie kolejnych stacji',
 '\n1. Kabaty\nFotograficzny spacer po Warszawie nad budowanym metrem zaczynamy w 1995 r. na Kabatach. To część miasta, która zmieniła się chyba najbardziej. Właściwie\xa0lepiej napisać, że współczesne Kabaty dopiero powstały na skutek budowy podziemnej kolei.\nDo lat 90. XX w. ciągnęły się tu pola i nieużytki aż do ściany Lasu Kabackiego. Dla warszawiaków przez lata były to odległe rubieże. Dziś to jedna z\xa0dzielnic uważanych za najlepsze do życia. Bliskość lasu i dojazd metrem do centrum w niewiele ponad 20 minut to dwa główne atuty.\xa0\xa0\n\n')

Ура, у нас получилось! И нам не нужен Селениум, чтобы делать редирект по понятному адресу. Реверс-инжиниринг помог.  
Правда, это не решает проблемы с тем, что нам не отдают новость полносьтю, но, кажется, для этого надо подписаться на ресурс. Вот жадины! Но тут уже без нарушения закона не обойтись, так что или довольствуемся первыми абзацами, или уходим на другой ресурс, менее жадный.   
Правда, там есть комментарии к статьям, тоже написанные по-польски. То есть можно махнуть рукой на новость и сделаьт коллекцию заголовков новостей, первых абзацев и комментариев к ним, что тотже неплохо.

### Сессия

Если вам необходимо ввести пароль, а потом работать как зарегистрированный пользователь, лучше использовать сессию, которая запомнит все данные. Об этом можно почитать [здесь](https://requests.readthedocs.io/en/latest/user/advanced/).

### Работа с бинарными файлами

Попробуем загрузить pdf-файл со статьей с сайта Киберленинка.

Правда, если мы будем загружать его все вместе с одного IP-адреса, то нас могут забанить за массовое скачивание.

In [53]:
pdf1 = requests.get("https://cyberleninka.ru/article/n/semantiko-sintaksicheskoe-opisanie-glagolov-professionalnoy-deyatelnosti-iz-materialov-k-semantiko-grammaticheskomu-slovaryu/pdf")

Можно заметить, что у объекта есть несколько полей, хранящих бинарные строки, например, `content` и `raw`. Это, собственно, содержимое загруженного файла.

Еще у него есть свойства на случай, если был загружен json (`json`) или текст (мы его уже использовали - `text`).

In [54]:
dir(pdf1)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Взглянем на содержимое загруженного файла. В первых байтах виден заголовок pdf-файла с автором, использованной программой и другими данными.

In [55]:
pdf1.content[:200]

b"%PDF-1.4\n%\xe2\xe3\xcf\xd3\n1 0 obj\n<</Author(katya)/CreationDate(D:20200129093126+03'00')/Creator(Adobe InDesign CC 14.0 \\(Macintosh\\))/ModDate(D:20200224020154+03'00')/Producer(Acrobat Distiller 15.0 \\(Macintosh"

Поставим себе библиотеку для чтения pdf-файлов и посмотрим на содержимое первой страницы.

In [27]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable


In [56]:
from pypdf import PdfReader
import io

Библиотека принимает на вход открытый файл, но при помощи `io.BytesIO()` мы сможем имитировать чтение из файла, которое будет брать данные из памяти.

In [57]:
f = io.BytesIO(pdf1.content)

reader = PdfReader(f)

Итак, посмотрим на текстовое содержимое первой страницы.

In [58]:
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [59]:
text

'УЧЕНЫЕ  ЗАПИСКИ  ПЕТРОЗАВОДСКОГО  ГОСУДАРСТВЕННОГО  УНИВЕРСИТЕТА\n© Смирнова  Е. А., 2020Т. 42. № 1. С. 72–81 Языкознание  2020\nDOI: 10.15393/uchz.art.2020.435УДК  81`1\nЕКАТЕРИНА  АНДРЕЕВНА  СМИРНОВА\nкандидат  филологических  наук , научный  сотрудник  секто -\nра теоретической  семантики\nФедеральное  государственное  бюджетное  учреждение  нау-\nки Институт  русского  языка  имени  В. В. Виноградова  РАН  \n(Москва , Российская  Федерация )\nkatarzina@yandex.ru\nСЕМАНТИКО -СИНТАКСИЧЕСКОЕ  ОПИСАНИЕ  ГЛАГОЛОВ  \nПРОФЕССИОНАЛЬНОЙ  ДЕЯТЕЛЬНОСТИ  \n(из материалов  к Семантико -грамматическому  словарю )*\nПредставлено  описание  парадигматических  возможностей  глагольной  группы  профессиональ -\nной деятельности  на -ничать  и выявлены  семантико -синтаксические  запреты  на образование  тех \nили иных  глагольных  форм . На материале  Национального  корпуса  русского  языка  (НКРЯ ) показа -\nны тенденции  в области  глагольного  словоизменения  и словообразования  внутри  данной  

М-да... С разбивкой на строки и абзацы полная беда. Попробуем хотя бы устранить преносы.

In [60]:
text.replace(' -\n', '').replace('-\n', '')

'УЧЕНЫЕ  ЗАПИСКИ  ПЕТРОЗАВОДСКОГО  ГОСУДАРСТВЕННОГО  УНИВЕРСИТЕТА\n© Смирнова  Е. А., 2020Т. 42. № 1. С. 72–81 Языкознание  2020\nDOI: 10.15393/uchz.art.2020.435УДК  81`1\nЕКАТЕРИНА  АНДРЕЕВНА  СМИРНОВА\nкандидат  филологических  наук , научный  сотрудник  сектора теоретической  семантики\nФедеральное  государственное  бюджетное  учреждение  науки Институт  русского  языка  имени  В. В. Виноградова  РАН  \n(Москва , Российская  Федерация )\nkatarzina@yandex.ru\nСЕМАНТИКО -СИНТАКСИЧЕСКОЕ  ОПИСАНИЕ  ГЛАГОЛОВ  \nПРОФЕССИОНАЛЬНОЙ  ДЕЯТЕЛЬНОСТИ  \n(из материалов  к Семантико -грамматическому  словарю )*\nПредставлено  описание  парадигматических  возможностей  глагольной  группы  профессиональной деятельности  на -ничать  и выявлены  семантико -синтаксические  запреты  на образование  тех \nили иных  глагольных  форм . На материале  Национального  корпуса  русского  языка  (НКРЯ ) показаны тенденции  в области  глагольного  словоизменения  и словообразования  внутри  данной  группы : \nзапо

### Потоковое аудио
Теперь попробуем работать с потоковым аудио. Я начал со страницы http://radio.garden , которая хранит ссылки на интернет-радиостанции. С ее помощью я вышел вот на этот адрес. Попробуем грузить с него потоковое аудио в файл.

In [61]:
# stream_url = 'https://radiorecord.hostingradio.ru/198096.aacp?listening-from-radio-garden=1680922353'
# stream_url = 'http://stream1.waszeradiofm.pl:8000/;?listening-from-radio-garden=1696354810'
# stream_url = 'http://stream4.nadaje.com:8052/;'
# stream_url = 'https://radio21.streamabc.net/radio21-buxtehude-mp3-192-4403353?sABC=662o19r4%230%234o3n4o6p107p0srs94636q7nr26pr465%23jro&aw_0_1st.playerid=web&amsparams=playerid:web;skey:1714100708'
# stream_url = 'https://ic1.smcdn.pl/2310-1.mp3'
stream_url = 'https://ic7.101.ru:8000/stream/reg/mp3/128/region_avto_36'

r = requests.get(stream_url, stream=True)

with open('stream.mp3', 'wb') as f:
    try:
        for block in r.iter_content(1024):
            f.write(block)
    except KeyboardInterrupt:
        pass

А попробуем теперь прослушать это аудио.

In [53]:
!pip install pydub

Defaulting to user installation because normal site-packages is not writeable


In [62]:
from pydub import AudioSegment
from pydub.playback import play

In [63]:
import io

In [65]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
stream_url = 'http://stream1.waszeradiofm.pl:8000/;?listening-from-radio-garden=1696354810'
r = requests.get(stream_url, stream=True)

try:
    for block in r.iter_content(14240):
        f = io.BytesIO(block)
        sound = AudioSegment.from_file(f)
        play(sound)
except KeyboardInterrupt:
    pass

ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd